# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [57]:
import optuna
from tsai.optuna import *
import wandb
api = wandb.Api()
import papermill as pm
from tsai.optuna import run_optuna_study

In [78]:
search_space = {
    'lookback': [12, 18, 24, 36, 48, 60]
}

In [79]:
def objective(trial:optuna.Trial):

    # Define search space here
    lookback = trial.suggest_int('lookback', low=12, high=60, step=6)

    # Call the training notebook using papermill (don't print the output)
    stdout_file = open('tmp/pm_stdout.txt', 'w')
    stderr_file = open('tmp/pm_stderr.txt', 'w')
    pm.execute_notebook(
        './training.ipynb',
        './tmp/pm_output.ipynb',
        parameters = {
            'config.lookback': lookback
        },
        stdout_file = stdout_file,
        stderr_file = stderr_file
    )

    # Close the output files
    stdout_file.close()
    stderr_file.close()

    # Get the output value of interest from the source notebook
    %store -r valid_loss
    return valid_loss
        

In [80]:
run_optuna_study(objective, study_type='gridsearch', search_space=search_space,
                  direction='minimize', path='tmp')

[I 2023-03-22 20:43:48,205] A new study created in memory with name: no-name-0f70d72e-e132-433f-af09-9f20950fede4
Black is not installed, parameters wont be formatted
Executing:  85%|██████████████████████████████████████████▋       | 29/34 [00:07<00:01,  3.65cell/s]
[W 2023-03-22 20:43:56,191] Trial 0 failed with parameters: {'lookback': 6} because of the following error: PapermillExecutionError(28, 21, 'cbs = L(WandbCallback(log_preds=False)) if config.use_wandb else L()\nlearn = TSForecaster(X, y, splits=splits, batch_size=config.bs, \n                     pipelines=[preproc_pipe, exp_pipe], arch="PatchTST", \n                     arch_config=dict(config.arch_config), \n                    cbs= cbs + ShowGraphCallback())\n#lr_max = learn.lr_find().valley\nlr_max = 1e-3\nprint(f"#params: {sum(p.numel() for p in learn.model.parameters())}")', 'RuntimeError', 'Trying to create tensor with negative dimension -1: [-1, 16]', ['\x1b----------------------------------------------------------

PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [21]":
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
Cell In[21], line 2
      1 cbs = L(WandbCallback(log_preds=False)) if config.use_wandb else L()
----> 2 learn = TSForecaster(X, y, splits=splits, batch_size=config.bs, 
      3                      pipelines=[preproc_pipe, exp_pipe], arch="PatchTST", 
      4                      arch_config=dict(config.arch_config), 
      5                     cbs= cbs + ShowGraphCallback())
      6 #lr_max = learn.lr_find().valley
      7 lr_max = 1e-3

File /usr/local/pip-global/tsai/tslearner.py:214, in TSForecaster.__init__(self, X, y, splits, tfms, inplace, sel_vars, sel_steps, weights, partial_n, train_metrics, valid_metrics, bs, batch_size, batch_tfms, pipelines, shuffle_train, drop_last, num_workers, do_set